# NLP Assignment: Language Modelling with Poetry (deadline **24.5.2024**)

This assignment will check your basic NLP understanding through the fundamental NLP task of **language modelling**.

You will reiterate on the task with techniques ranging from simple n-gram counting to embeddings and deep learning.


You will work with the same short poetry texts that should be very familiar to you by now:

In [ ]:
!wget -nc https://raw.githubusercontent.com/GustikS/smu-nlp/master/robert_frost.txt
!wget -nc https://raw.githubusercontent.com/GustikS/smu-nlp/master/edgar_allan_poe.txt

Unless stated otherwise (Ex. 4,5,6), work just with the Robert Frost file for simplicity (Ex. 1,2,3,7,8)

In [ ]:
!head  

#### **Excercise 1**: Markov Language Model (**2 points**)

1. Create a 1st order Markov (bi-gram) language model
  - work with matrix representation of the model
    - i.e. not dictionaries as we did in the tutorial!
    - hence you'll need to assign indices to the words, too
      - include an extra \<UNK\> token for unseen words
  - correctly handle beginnings and ends of sentences
    - sentence = line (skip empty lines)
  - lower case and properly tokenize your sentences
    - but skip all other text preprocessing

In [ ]:
import numpy as np

def create_markov_model(filename):
    """
    Creates a 1st order Markov model from a text file using matrix representation.

    Args:
        filename: Path to the text file.

    Returns:
        A tuple containing:
            - word_to_index: Dictionary mapping words to unique integer indices.
            - transition_matrix: A 2D numpy array representing transition probabilities.
            - unk_index: The index assigned to the <UNK> token.
    """
    # Build word dictionary and assign unique indices
    word_to_index = {"<UNK>": 0, "<s>": 1, "</s>": 2}  # Include <s> and </s>
    index_to_word = ["<UNK>", "<s>", "</s>"]  # Reverse mapping
    word_count = len(word_to_index)

    with open(filename) as f:
        for line in f:
            tokens = line.rstrip().lower().split()
            if not tokens:
                continue
            for word in tokens:
                if word not in word_to_index:
                    word_to_index[word] = word_count
                    index_to_word.append(word)
                    word_count += 1

    # Initialize transition matrix with zeros
    vocab_size = len(word_to_index)
    transition_matrix = np.zeros((vocab_size, vocab_size))

    # Process text to build transition counts
    unk_index = word_to_index["<UNK>"]
    with open(filename) as f:
        for line in f:
            tokens = line.rstrip().lower().split()
            if not tokens:
                continue
            tokens = ['<s>'] + tokens + ['</s>']  # Add start and end tokens

            # Use loop variable for iteration
            for i in range(len(tokens) - 1):
                prev_word_index = word_to_index.get(tokens[i], unk_index)
                next_word_index = word_to_index.get(tokens[i + 1], unk_index)
                transition_matrix[prev_word_index, next_word_index] += 1

    # Normalize transition counts to probabilities
    for i in range(vocab_size):
        total_count = np.sum(transition_matrix[i])
        if total_count > 0:
            transition_matrix[i] /= total_count

    return word_to_index, transition_matrix, unk_index

# Example usage
word_to_index, transition_matrix, unk_index = create_markov_model("robert_frost.txt")

word1 = "the"
word2 = "young"
print(f"Transition probability from '{word1}' to '{word2}': {transition_matrix[word_to_index[word1], word_to_index.get(word2, unk_index)]}")


#### **Excercise 2**: Probability + Smoothing (**1 points**)
1. write a function to obtain probability of a given sentence with your model
    - include the beginning and end mark of the sentence as well
    - test some sentences and assure the probability makes sense
2. incorporate the add-1 (Laplace) smoothing to account for unseen bi-grams
    - you should have your \<UNK\> for unseen unigrams already

In [ ]:
def sentence_probability(sentence, word_to_index, transition_matrix, unk_index):
    """
    Computes the probability of a sentence using the Markov model.

    Args:
        sentence: The sentence for which to compute the probability.
        word_to_index: Dictionary mapping words to unique integer indices.
        transition_matrix: A 2D numpy array representing transition probabilities.
        unk_index: The index assigned to the <UNK> token.

    Returns:
        The probability of the sentence.
    """
    tokens = sentence.lower().split()
    prob = 1.0

    

    for i in range(len(tokens) - 1):
        prev_word_index = word_to_index.get(tokens[i], unk_index)
        next_word_index = word_to_index.get(tokens[i + 1], unk_index)
        prob *= transition_matrix[prev_word_index, next_word_index]

    return prob

def laplace_smoothing(transition_matrix):
    """
    Applies Laplace smoothing to the transition matrix.

    Args:
        transition_matrix: A 2D numpy array representing transition counts.

    Returns:
        A 2D numpy array representing smoothed transition probabilities.
    """
    vocab_size = transition_matrix.shape[0]
    smoothed_matrix = transition_matrix + 1  # Add 1 to each count

    # Normalize with added vocabulary size
    for i in range(vocab_size):
        total_count = np.sum(smoothed_matrix[i])
        smoothed_matrix[i] /= total_count

    return smoothed_matrix

# Example usage
sentence = "<s> the young folk held some hope out to each other. </s>"
print(f"Sentence probability: {sentence_probability(sentence, word_to_index, transition_matrix, unk_index)}")

transition_matrix_smoothed = laplace_smoothing(transition_matrix)
print(f"Sentence probability with Laplace smoothing: {sentence_probability(sentence, word_to_index, transition_matrix_smoothed, unk_index)}")


#### **Excercise 3**: Perplexity (**1 points**)
1. write a function fo calculate perplexity of your smoothed model on a given sentence
  - including its beginning and ending
2. find the sentence(s) from the corpus with minimum and maximum perplexity

In [ ]:
def calculate_perplexity(sentence, word_to_index, transition_matrix, unk_index):
    """
    Calculates the perplexity of a given sentence using the Markov model.

    Args:
        sentence: The input sentence as a string.
        word_to_index: Dictionary mapping words to unique integer indices.
        transition_matrix: A 2D numpy array representing transition probabilities.
        unk_index: The index assigned to the <UNK> token.

    Returns:
        The perplexity of the sentence.
    """
    tokens = sentence.lower().split()
    length = len(tokens)
    ps = []

    for i in range(length - 1):
        ps.append(transition_matrix[word_to_index.get(tokens[i], unk_index), word_to_index.get(tokens[i + 1], unk_index)])

    val = 1
    for p in ps:
        val *= 1/p
    val = val**(1/length)
    return val

def find_min_max_perplexity_sentences(filename, word_to_index, transition_matrix, unk_index):
    """
    Finds the sentences with the minimum and maximum perplexity from the corpus.

    Args:
        filename: Path to the text file.
        word_to_index: Dictionary mapping words to unique integer indices.
        transition_matrix: A 2D numpy array representing transition probabilities.
        unk_index: The index assigned to the <UNK> token.

    Returns:
        A tuple containing:
            - Sentence with minimum perplexity
            - Sentence with maximum perplexity
            - Minimum perplexity value
            - Maximum perplexity value
    """
    min_perplexity = float('inf')
    max_perplexity = float('-inf')
    min_sentence = ""
    max_sentence = ""

    with open(filename) as f:
        for line in f:
            tokens = line.rstrip().lower().split()
            if not tokens:
                continue
            sentence = ' '.join(tokens)
            perplexity = calculate_perplexity(sentence, word_to_index, transition_matrix, unk_index)
            if perplexity < min_perplexity:
                min_perplexity = perplexity
                min_sentence = sentence
            if perplexity > max_perplexity:
                max_perplexity = perplexity
                max_sentence = sentence

    return min_sentence, max_sentence, min_perplexity, max_perplexity

# Calculate the perplexity of a sentence
sentence = "the young folk held some hope out to each other. </s>"
perplexity = calculate_perplexity(sentence, word_to_index, transition_matrix, unk_index)
print(f"Perplexity of sentence '{sentence}': {perplexity}")

# Find the sentences with minimum and maximum perplexity from the corpus
min_sentence, max_sentence, min_perplexity, max_perplexity = find_min_max_perplexity_sentences("robert_frost.txt", word_to_index, transition_matrix, unk_index)
print(f"Sentence with minimum perplexity: '{min_sentence}' (Perplexity: {min_perplexity})")
print(f"Sentence with maximum perplexity: '{max_sentence}' (Perplexity: {max_perplexity})")

#### **Excercise 4:**  Markov classifier (**4 points**)
Implement your own probabilistic classifier based on your bi-gram language model. That is:
  1. given some classes of sentences, train a separate language model for each class respectively
  2. then classify a given sentence (=sample) based on maximum a-posteriori probability (MAP)
    - i.e. don't forget about the class priors, too!
    - use log probabilities!
    - make sure your smoothing treats all the classes equally!
      - ...think about what happens to the UNK token
  3. evaluate on a task of recognizing sentences from the 2 different poets (Frost vs. Poe)
    - split the sentences (=samples) from each poet into train-test in advance!
      - train-test split 70:30
        - do not shuffle sentences
      - skip empty lines (of course)
 	- report all accuracy values + a confusion matrix

*Note that this is very similar to our previous classification with Naive Bayes, but with bi-grams instead of unigrams.*

In [ ]:

from sklearn.metrics import accuracy_score, confusion_matrix

def classify_sentence(sentence, frost_model, poe_model, frost_prior, poe_prior):
    frost_log_prob = sentence_probability(sentence, *frost_model) + np.log(frost_prior)
    poe_log_prob = sentence_probability(sentence, *poe_model) + np.log(poe_prior)

    return "Frost" if frost_log_prob > poe_log_prob else "Poe"

def load_data(filename):
    with open(filename) as f:
        return [line.strip() for line in f if line.strip()]

def split_data(data, train_ratio=0.7):
    train_size = int(len(data) * train_ratio)
    return data[:train_size], data[train_size:]

# Load and split data
frost_data = load_data("robert_frost.txt")
poe_data = load_data("edgar_allan_poe.txt")

frost_train, frost_test = split_data(frost_data)
poe_train, poe_test = split_data(poe_data)

# Save training data for model creation
with open("frost_train.txt", "w") as f:
    f.write("\n".join(frost_train))
with open("poe_train.txt", "w") as f:
    f.write("\n".join(poe_train))

# Train models
frost_model = create_markov_model("frost_train.txt")
poe_model = create_markov_model("poe_train.txt")

# Apply Laplace smoothing
frost_model = (frost_model[0], laplace_smoothing(frost_model[1]), frost_model[2])
poe_model = (poe_model[0], laplace_smoothing(poe_model[1]), poe_model[2])

# Class priors
frost_prior = len(frost_train) / (len(frost_train) + len(poe_train))
poe_prior = len(poe_train) / (len(frost_train) + len(poe_train))

# Classify test sentences
y_true = ["Frost"] * len(frost_test) + ["Poe"] * len(poe_test)
y_pred = []

for sentence in frost_test:
    y_pred.append(classify_sentence(sentence, frost_model, poe_model, frost_prior, poe_prior))

for sentence in poe_test:
    y_pred.append(classify_sentence(sentence, frost_model, poe_model, frost_prior, poe_prior))

# Evaluation
accuracy = accuracy_score(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred, labels=["Frost", "Poe"])

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")

#### **Excercise 5**: PPMI word-word cooccurence (**2 points**)
1. Create a word-word co-occurence matrix from all the text of both the poets altogether
  - use a sliding window of size 5 (2 left + 2 right context words)
    - remember that you can reuse existing solutions...
2. Post-process the matrix with PPMI

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import pandas as pd
import numpy as np

def co_occurrence(texts, context_size):
    window_size = (context_size - 1) // 2
    cooccurrence = {}
    vocab = set()
    
    for text in texts:
        tokens = word_tokenize(text)
        for token in tokens:
            vocab.add(token)

        # Sliding window
        for i in range(window_size, len(tokens) - window_size):
            token = tokens[i]
            context = tokens[(i - window_size):i] + tokens[(i + 1):(i + 1 + window_size)]
            for t in context:
                key = tuple(sorted([t, token]))
                cooccurrence[key] = cooccurrence.get(key, 0) + 1

    # Formulate the dictionary into a dataframe
    vocab = sorted(vocab)
    df = pd.DataFrame(data=np.zeros((len(vocab), len(vocab)), dtype=np.float64), index=vocab, columns=vocab)
    for key, value in cooccurrence.items():
        df.at[key[0], key[1]] = value
        df.at[key[1], key[0]] = value
    return df

def ppmi(df, positive=True):
    col_totals = df.sum(axis=0)
    row_totals = df.sum(axis=1)
    total = col_totals.sum()
    expected = np.outer(row_totals, col_totals) / total
    df = df / expected
    df = np.log(df)

    df[np.isinf(df)] = 0.0  # log(0) = 0  ; or silence distracting warnings about log(0) with np.errstate(divide='ignore'):
    # ppmi
    if positive:
        df[df < 0] = 0.0
    return df

# Reading text files
with open('edgar_allan_poe.txt', 'r') as file:
    poe_text = file.read()

with open('robert_frost.txt', 'r') as file:
    frost_text = file.read()

# Combine the texts
texts = [poe_text, frost_text]
context_size = 3

# Generate the co-occurrence matrix
co_occurrence_matrix = co_occurrence(texts, context_size)

# Compute the PPMI matrix using the provided function
ppmi_matrix = ppmi(co_occurrence_matrix)

# Display the PPMI matrix
print(ppmi_matrix)

#### **Excercise 6**: Word embeddings (**1 points**)
1. Extract 8-dimensional word embeddings from your PPMI matrix
  - using the truncated SVD matrix decomposition
2. Plot them in 2D with word labels

In [ ]:
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt


svd = TruncatedSVD(n_components=8)
word_embeddings = svd.fit_transform(ppmi_matrix)

# Further reduce the dimensionality to 2 dimensions for plotting
svd_2d = TruncatedSVD(n_components=2)
word_embeddings_2d = svd_2d.fit_transform(word_embeddings)

# Plotting the 2D embeddings with word labels
plt.figure(figsize=(14, 10))
plt.scatter(word_embeddings_2d[:, 0], word_embeddings_2d[:, 1], marker='o')

for i, word in enumerate(ppmi_matrix.index):
    plt.annotate(word, (word_embeddings_2d[i, 0], word_embeddings_2d[i, 1]), fontsize=12)

plt.title('2D Word Embeddings from PPMI Matrix')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.grid(True)
plt.show()

#### **Excercise 7:** LSTM Language Model (**3 points**)
1. Formulate a proper dataset for language modelling on longer sequences from the text
  - beyond the n-gram scope, use 10 consecutive words
2. Create a suitable LSTM-based language model
3. Initialize the embedding layer of your model with your "pretrained" SVD embeddings
4. Train the model in a suitable fashion

In [ ]:

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
from nltk.tokenize import word_tokenize


# Read and combine texts
with open('edgar_allan_poe.txt', 'r') as file:
    poe_text = file.read()

with open('robert_frost.txt', 'r') as file:
    frost_text = file.read()

texts = poe_text + ' ' + frost_text

# Tokenize the text
tokens = word_tokenize(texts.lower())

# Create sequences of 10 words
sequence_length = 10
sequences = []

for i in range(sequence_length, len(tokens)):
    seq = tokens[i-sequence_length:i+1]
    sequences.append(seq)

# Prepare tokenizer (mapping from word to index)
vocab = list(set(tokens))
word2idx = {word: idx for idx, word in enumerate(vocab)}
sequences = [[word2idx[word] for word in seq] for seq in sequences]

# Split sequences into input (X) and output (y)
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a custom dataset class
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create DataLoader instances
train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, embeddings=None):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        if embeddings is not None:
            self.embedding.weight = nn.Parameter(torch.tensor(embeddings, dtype=torch.float32))
            self.embedding.weight.requires_grad = False  # Optionally freeze the embedding layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, dropout=0.2, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return x

vocab_size = len(vocab)
embedding_dim = 8
hidden_dim = 100
output_dim = vocab_size

model = LSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim)


 #### **Excercise 8**: Sampling (**1 point**)
1. Sample some text from your models w.r.t. to the output (next) word probability distribution
  - for both bigram and LSTM models - which is better?

2. Paste your best "poem" here:

# **Notes**

If this seems like a lot of work, reiterate through the tutorials (which is the purpose, anyway) - we did most of this already!
  - i.e., I don't expect you to be overly creative, you can find all the pieces in the tutorial notebooks
  - but feel free to write a more efficient/clean/suitable code from scratch
  - only the libraries we used in the tutorials are allowed
  - teamwork is forbidden!
  - chatGPT is allowed

Before submitting:
  - make sure your code is runnable (in Colab)!
  - comment your code at least a bit, make it readable!
  - add printouts after every excercise
  - submit just the notebook as a single file (filename = username)


I will only evaluate *correctness* of the solutions, not quality of the models
  - i.e. no need to spent too much time with text preprocessing or hyperparameter tuning (LSTM)

# **Bonus points**
You can recover some of the potentially lost points here, but you can't reach more than 15 points in total from this assignment
  - *explicitly mark the bonus points you consider as fullfilled!*
<br/><br/>

1. (1p) Use a 2nd order Markov model with a sparse tensor instead
  - i.e., do Laplace smoothing without actually modifying the model (tensor)

2. (1p) In your LSTM model, do not start every sample sequence with an empty hidden state (zeros), but pass the last hidden state from the preceding sample sequence
  - i.e. the sequence ending just before the start of the current sequence
  - but do not propagate gradient into the previous sample (detach the state value from the backprop graph)

3. (1p) Improve your text generation by sampling from the top-p next words only

4. (2p) Use a different (larger) textual corpus of poems
  - e.g. Corpus of Czech Verse, collected at the Institute of Czech Literature at Czech Academy of Sciences
    - available at https://github.com/versotym/corpusCzechVerse
    - just be careful about czech character encoding

5. (10p) An alternative task - create a "whisperer" (bot player) for the popular game ["Kryci jmena"](https://krycijmena.cz/) based on word embeddings
  - i.e. automatically search for words that would cover (be related/close to) subsets of the "positive" words, while trying to avoid the "negative" words

In [ ]:
# Corpus of Czech Verse
!git clone https://github.com/versotym/corpusCzechVerse

In [ ]:
import json
from os import listdir
path = 'corpusCzechVerse/ccv/'

books = []
files = [f for f in listdir(path)]
for i, f in enumerate(sorted(files)):
    print(f)
    book = open(path+f)
    books.append(json.load(book))

    break

In [ ]:
i=0
book = books[i]

for poem in book:
    author = poem['p_author']['name']
    title = poem['biblio']['p_title']
    print('------' + title + '------\n')
    body = poem['body']
    for text in body:
        for line in text:
            print(line['text'])
        print('')
    break